# Naives Bayes (get a probability of a comment being 'sarcasm' or 'bullying' given 'a word'  is in this comment)

In [2]:
import numpy as np 
import pandas as pd
from nltk.corpus import stopwords 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score, accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix

## Fetching the dataset

In [2]:
merged_df = pd.read_csv('merged_all_csv.csv')

In [3]:
merged_df

,Unnamed: 0,label,comment,comment_cleaned,Words Per Tweet
0,44,sarcasm,wow it is totally unreasonable to assume that ...,wow it is totally unreasonable to assume that ...,25.0
1,443,sarcasm,A WHITE FUCKIN MALE is out to destroy 2 strong...,"a WHITE FUCKIN MALE is out to destroy strong,...",18.0
2,807,sarcasm,He probably just didn't want to read them beca...,he probably just didn't want to read them beca...,23.0
3,1745,sarcasm,It's like he's saying we're entering a post ge...,it's like he's saying we're entering a post ge...,21.0
4,2022,sarcasm,But it was clearly only dismissed because of d...,but it was clearly only dismissed because of d...,19.0
...,...,...,...,...,...
42497,47687,ethnicity,"Black ppl aren't expected to do anything, depe...","black ppl aren't expected to do anything, depe...",42.0
42498,47688,ethnicity,Turner did not withhold his disappointment. Tu...,turner did not withhold his disappointment. Tu...,45.0
42499,47689,ethnicity,I swear to God. This dumb nigger bitch. I have...,i swear to God. This dumb nigger bitch. I have...,20.0
42500,47690,ethnicity,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,yea fuck you RT IF YOURE A NIGGER FUCKING UNF...,15.0


In [4]:
merged_df = merged_df[['label', 'comment_cleaned', 'Words Per Tweet']]
merged_df.dtypes

label               object
comment_cleaned     object
Words Per Tweet    float64
dtype: object

In [5]:
merged_df['comment_cleaned'] = merged_df['comment_cleaned'].astype(str)

## Lowercasing the dataset

In [6]:
merged_df['comment_cleaned_lower'] = merged_df['comment_cleaned'].str.lower()
merged_df

,label,comment_cleaned,Words Per Tweet,comment_cleaned_lower
0,sarcasm,wow it is totally unreasonable to assume that ...,25.0,wow it is totally unreasonable to assume that ...
1,sarcasm,"a WHITE FUCKIN MALE is out to destroy strong,...",18.0,"a white fuckin male is out to destroy strong,..."
2,sarcasm,he probably just didn't want to read them beca...,23.0,he probably just didn't want to read them beca...
3,sarcasm,it's like he's saying we're entering a post ge...,21.0,it's like he's saying we're entering a post ge...
4,sarcasm,but it was clearly only dismissed because of d...,19.0,but it was clearly only dismissed because of d...
...,...,...,...,...
42497,ethnicity,"black ppl aren't expected to do anything, depe...",42.0,"black ppl aren't expected to do anything, depe..."
42498,ethnicity,turner did not withhold his disappointment. Tu...,45.0,turner did not withhold his disappointment. tu...
42499,ethnicity,i swear to God. This dumb nigger bitch. I have...,20.0,i swear to god. this dumb nigger bitch. i have...
42500,ethnicity,yea fuck you RT IF YOURE A NIGGER FUCKING UNF...,15.0,yea fuck you rt if youre a nigger fucking unf...


## Getting the most frequent words

The most frequent words are for sure going to be stop words, so in order to find frequent occurences of meaningful words, let's create a new column which will contain the comment without the stopwords.


This step is part of EDA - can skip this section

In [7]:
stop_words = set(stopwords.words('english')) 
merged_df['text_without_stop_words'] = merged_df['comment_cleaned_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

merged_df

,label,comment_cleaned,Words Per Tweet,comment_cleaned_lower,text_without_stop_words
0,sarcasm,wow it is totally unreasonable to assume that ...,25.0,wow it is totally unreasonable to assume that ...,wow totally unreasonable assume agency covered...
1,sarcasm,"a WHITE FUCKIN MALE is out to destroy strong,...",18.0,"a white fuckin male is out to destroy strong,...","white fuckin male destroy strong, independent ..."
2,sarcasm,he probably just didn't want to read them beca...,23.0,he probably just didn't want to read them beca...,probably want read illegal read news media see...
3,sarcasm,it's like he's saying we're entering a post ge...,21.0,it's like he's saying we're entering a post ge...,like he's saying we're entering post gender so...
4,sarcasm,but it was clearly only dismissed because of d...,19.0,but it was clearly only dismissed because of d...,clearly dismissed death threats even though li...
...,...,...,...,...,...
42497,ethnicity,"black ppl aren't expected to do anything, depe...",42.0,"black ppl aren't expected to do anything, depe...","black ppl expected anything, depended anything..."
42498,ethnicity,turner did not withhold his disappointment. Tu...,45.0,turner did not withhold his disappointment. tu...,turner withhold disappointment. turner called ...
42499,ethnicity,i swear to God. This dumb nigger bitch. I have...,20.0,i swear to god. this dumb nigger bitch. i have...,swear god. dumb nigger bitch. got bleach hair ...
42500,ethnicity,yea fuck you RT IF YOURE A NIGGER FUCKING UNF...,15.0,yea fuck you rt if youre a nigger fucking unf...,"yea fuck rt youre nigger fucking unfollow me, ..."


In [8]:
unmerged_df_sarcasm = merged_df[merged_df['label'] == 'sarcasm']

In [9]:
# first get the strings from the dataframe column
strings = unmerged_df_sarcasm['text_without_stop_words'].tolist()

# create a list to store all words, loop through the strings and add each word to the list
all_words = []
for string in strings:
    words = str(string).split()
    all_words.extend(words)

# get the 10, 20,30 most frequent words using collections.Counter
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(20)

#print them, we just do all this to have an intuition of what word(s) to pick to run our naives bayes tests
print("Here are the 20 most frequent words in the sarcastic comments:")
for word, count in most_common_words:
    print(f"{word}: {count}")

Here are the 20 most frequent words in the sarcastic comments:
like: 1000
people: 895
would: 751
get: 729
i'm: 599
yeah,: 547
one: 544
know: 542
yeah: 523
sure: 464
make: 436
well: 419
good: 384
can't: 381
want: 335
way: 332
going: 331
think: 329
see: 325
really: 323


In [10]:
# first get the strings from the dataframe column
strings = merged_df['text_without_stop_words'].tolist()

# create a list to store all words, loop through the strings and add each word to the list
all_words = []
for string in strings:
    words = str(string).split()
    all_words.extend(words)

# get the 10, 20,30 most frequent words using collections.Counter
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(20)

#print them, we just do all this to have an intuition of what word(s) to pick to run our naives bayes tests
print("Here are the 20 most frequent words in the bully comments:")
for word, count in most_common_words:
    print(f"{word}: {count}")

Here are the 20 most frequent words in the bully comments:
school: 6816
like: 5791
dumb: 5112
fuck: 5108
high: 5026
people: 4394
bullied: 4153
u: 3654
rape: 3518
one: 3241
gay: 3225
get: 2874
nigger: 2837
&: 2549
rt: 2546
ass: 2471
girls: 2458
black: 2449
would: 2364
girl: 2364


## Labelling the dataset 1 for cyberbullying, 0 for sarcasm, declaring our X and y (features and target) & performing the train-test split

In [11]:
merged_df['label'] = np.where(merged_df['label'] == 'sarcasm', 0, 1)

In [12]:
merged_df

,label,comment_cleaned,Words Per Tweet,comment_cleaned_lower,text_without_stop_words
0,0,wow it is totally unreasonable to assume that ...,25.0,wow it is totally unreasonable to assume that ...,wow totally unreasonable assume agency covered...
1,0,"a WHITE FUCKIN MALE is out to destroy strong,...",18.0,"a white fuckin male is out to destroy strong,...","white fuckin male destroy strong, independent ..."
2,0,he probably just didn't want to read them beca...,23.0,he probably just didn't want to read them beca...,probably want read illegal read news media see...
3,0,it's like he's saying we're entering a post ge...,21.0,it's like he's saying we're entering a post ge...,like he's saying we're entering post gender so...
4,0,but it was clearly only dismissed because of d...,19.0,but it was clearly only dismissed because of d...,clearly dismissed death threats even though li...
...,...,...,...,...,...
42497,1,"black ppl aren't expected to do anything, depe...",42.0,"black ppl aren't expected to do anything, depe...","black ppl expected anything, depended anything..."
42498,1,turner did not withhold his disappointment. Tu...,45.0,turner did not withhold his disappointment. tu...,turner withhold disappointment. turner called ...
42499,1,i swear to God. This dumb nigger bitch. I have...,20.0,i swear to god. this dumb nigger bitch. i have...,swear god. dumb nigger bitch. got bleach hair ...
42500,1,yea fuck you RT IF YOURE A NIGGER FUCKING UNF...,15.0,yea fuck you rt if youre a nigger fucking unf...,"yea fuck rt youre nigger fucking unfollow me, ..."


In [13]:
X = merged_df['comment_cleaned_lower']
y = merged_df['label']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [15]:
X_train.shape, X_test.shape

((29751,), (12751,))

In [16]:
print(X_test[34464], y_test[34464])

if i saw this type of person in school i would bully them too 1


Let's use this comment as a prediction example later on, as it is pretty clear that it is bullying.

## Defining our baseline accuracy

In [17]:
y_test.value_counts()

1    9599
0    3152
Name: label, dtype: int64

In [18]:
baseline_accuracy = (2435/(y_test.value_counts().sum()))
print('Baseline accuracy score: {0:0.4f}'. format(baseline_accuracy))
#our baseline accuracy is quite low because we took care of having balanced classes

Baseline accuracy score: 0.1910


## Training a Multinomial Naives Bayes model

In [19]:
# Vectorize text reviews to numbers
vec = CountVectorizer(stop_words='english')
#vec = TfidfVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [20]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [21]:
model.score(X_test[:300], y_test[:300])
#interesting in this dataset, with TfidfVectorizer accuracy =0.81, with CountVectorizer accuracy=0.92

0.92

In [22]:
#making a prediction on a sentence within the test set
model.predict(vec.transform(['if i saw this type of person in school i would bully them too']))
#correct predicted label

array([1])

Correct predicted label (1 = bullying)